In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

# CGV

## Event List

In [2]:
def get_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        movie_name = m.group()[1:-1]
        goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [3]:
event_df = get_EventList(goods_filter="TTT")
event_df

,event_idx,goods_idx,movie,goods,period
0,202403253872,2024032538723859,고질라 X 콩-뉴 엠파이어,TTT,2024.03.27 ~ 2024.04.26
5,202403183820,2024031838203819,극장판 스파이 패밀리 코드-화이트,TTT,2024.03.20 ~ 2024.04.19
6,202404013900,2024040139003887,기동전사 건담 시드 프리덤,TTT,2024.04.03 ~ 2024.05.02
15,202404093946,2024040939463943,슈가│어거스트 디 투어~,TTT 증정,2024.04.10 ~ 2024.04.16
25,202404013901,2024040139013888,유미의 세포들 더 무비,TTT,2024.04.03 ~ 2024.05.02
28,202404083925,2024040839253922,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
32,202403043767,2024030437673750,패스트 라이브즈,TTT,2024.03.06 ~ 2024.04.21


## Goods

- type2: 재고보유가능성높음
- type3: 재고소진임박
- type4: 마감

In [4]:
def get_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [5]:
giveawayIndex = '2024032538723859'
Theaters = get_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강남,0056,type2
1,CGV 강변,0001,type4
2,CGV 구로,0010,type2
3,CGV 동대문,0252,type2
4,CGV 등촌,0230,type2
...,...,...,...
59,CGV 서전주,0225,type2
60,CGV 전주고사,0213,type4
61,CGV 목포평화광장,0280,type2
62,CGV 순천신대,0268,type4
